# **IP4_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals:
You have a target of selling 5500 units.
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target.
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours:
You have both full-time and part-time employees.
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours.
Similarly, like sales, there could be deviations in these working hours.

Overtime:
Additionally, your employees might end up working overtime.
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you.
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities.
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Parameters:
- S: Sales target (5500 units)
- Rf: Selling rate for full-time employees (5 records/hour)
- Rp: Selling rate for part-time employees (2 records/hour)
- Hf: Full time employees total expected working hours (800 hours)
- Hp: Part time employees total expected working hours (320 hours)
- O: Overtime target for full-time employees (100 hours)
- P1, P2, P3, P4: Priority factors

Decision variables:
- X1: Number of hours full time employees work
- X2: Number of hours part time employees work
- X3: Number of overtime hours for full-time employees work
- X4: Number of overtime hours for part-time employees work
- X5: Number of units sold by full-time employees
- X6: Number of units sold by part-time employees

Objective function:
Minimize Z = P1*X1 + 2*P2*X2 + P3*X3 + 3*P4*X4 - P1*S + P2*(Hf+Hp) + P3*O 

Given the constraints:
- X5/Rf <= X1 <= Hf 
- X6/Rp <= X2 <= Hp 
- 0 <= X3 <= O 
- X4 >= 0
- X5 + X6 = S 

Explanation: 
The objective function is the sum of the weighted deviatio

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response2.choices[0].message.content)

```
from pyomo.environ import *

# Create Model
model = ConcreteModel()

# Define sets
model.I = Set(initialize=['FTE', 'PTE'])

# Parameters Initialized with example data
model.S = Param(initialize=5500)
model.Rf = Param(initialize=5)
model.Rp = Param(initialize=2)
model.Hf = Param(initialize=800)
model.Hp = Param(initialize=320)
model.O = Param(initialize=100)
model.P1 = Param(initialize=1.0)
model.P2 = Param(initialize=2.0)
model.P3 = Param(initialize=3.0)
model.P4 = Param(initialize=4.0)

# Decision Variables
model.X1 = Var(within=NonNegativeReals) #working hours by full time
model.X2 = Var(within=NonNegativeReals) #working hours by part time
model.X3 = Var(within=NonNegativeReals) #overtime hours by full time
model.X4 = Var(within=NonNegativeReals) #overtime hours by part time
model.X5 = Var(within=NonNegativeReals) #units sold by full-time workers
model.X6 = Var(within=NonNegativeReals) #units sold by part-time workers

# Objective
model.obj = Objective(expr = model.P1 * model.X1

### **Run the code Generated by GPT4**

In [9]:
from pyomo.environ import *

# Create Model
model = ConcreteModel()

# Define sets
model.I = Set(initialize=['FTE', 'PTE'])

# Parameters Initialized with example data
model.S = Param(initialize=5500)
model.Rf = Param(initialize=5)
model.Rp = Param(initialize=2)
model.Hf = Param(initialize=800)
model.Hp = Param(initialize=320)
model.O = Param(initialize=100)
model.P1 = Param(initialize=1.0)
model.P2 = Param(initialize=2.0)
model.P3 = Param(initialize=3.0)
model.P4 = Param(initialize=4.0)

# Decision Variables
model.X1 = Var(within=NonNegativeReals) #working hours by full time
model.X2 = Var(within=NonNegativeReals) #working hours by part time
model.X3 = Var(within=NonNegativeReals) #overtime hours by full time
model.X4 = Var(within=NonNegativeReals) #overtime hours by part time
model.X5 = Var(within=NonNegativeReals) #units sold by full-time workers
model.X6 = Var(within=NonNegativeReals) #units sold by part-time workers

# Objective
model.obj = Objective(expr = model.P1 * model.X1 + 2 * model.P2 * model.X2 + model.P3 * model.X3 + 3 * model.P4 * model.X4 - model.P1 * model.S + model.P2 * (model.Hf + model.Hp) + model.P3 * model.O,
                      sense= minimize)

# Constraints
model.constraint1 = Constraint(expr = model.X5/model.Rf <= model.X1 <= model.Hf)
model.constraint2 = Constraint(expr = model.X6/model.Rp <= model.X2 <= model.Hp)
model.constraint3 = Constraint(expr = 0 <= model.X3 <= model.O)
model.constraint4 = Constraint(expr = model.X4 >= 0)
model.constraint5 = Constraint(expr = model.X5 + model.X6 == model.S)

# Solve Model
SolverFactory('glpk').solve(model).write()

# model display
model.pprint()


PyomoException: Cannot convert non-constant Pyomo expression (0.2*X5  <=  X1) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [22]:
from pyomo.environ import *

# Create Model
model = ConcreteModel()


# Parameters Initialized with example data
model.S = Param(initialize=5500)
model.Rf = Param(initialize=5)
model.Rp = Param(initialize=2)
model.Hf = Param(initialize=800)
model.Hp = Param(initialize=320)
model.O = Param(initialize=100)
model.P1 = Param(initialize=4)
model.P2 = Param(initialize=3)
model.P3 = Param(initialize=2)
model.P4 = Param(initialize=1)

# Decision Variables
model.X1 = Var(within=NonNegativeReals) #working hours by full time
model.X2 = Var(within=NonNegativeReals) #working hours by part time
model.X3 = Var(within=NonNegativeReals) #overtime hours by full time
model.X4 = Var(within=NonNegativeReals) #overtime hours by part time
model.X5 = Var(within=NonNegativeReals) #units sold by full-time workers
model.X6 = Var(within=NonNegativeReals) #units sold by part-time workers

# Objective
model.obj = Objective(expr = model.P1 * model.X1 + 2 * model.P2 * model.X2 + model.P3 * model.X3 + 3 * model.P4 * model.X4 - model.P1 * model.S + model.P2 * (model.Hf + model.Hp) + model.P3 * model.O,
                      sense= minimize)

# Constraints
model.constraint1_lower = Constraint(
    expr= model.X5 / model.Rf <= model.X1)

model.constraint1_upper = Constraint(
    expr= model.X1 <= model.Hf)

# Constraints for model.constraint2
model.constraint2_lower = Constraint(
    expr= model.X6 / model.Rp <= model.X2)

model.constraint2_upper = Constraint(
    expr= model.X2 <= model.Hp)
model.constraint3_lower = Constraint(
    expr= 0 <= model.X3)

# Upper bound constraint
model.constraint3_upper = Constraint(
    expr= model.X3 <= model.O)

model.constraint4 = Constraint(expr = model.X4 >= 0)
model.constraint5 = Constraint(expr = model.X5 + model.X6 == model.S)

# Solve Model
SolverFactory('glpk').solve(model)

# model display
model.pprint()


10 Param Declarations
    Hf : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :   800
    Hp : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :   320
    O : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :   100
    P1 : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     4
    P2 : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     3
    P3 : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     2
    P4 : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     1
    Rf : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     5
    Rp : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
 